# VA10

# Notebook set-up

In [219]:
import os 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sb
import io
import statistics as stat
import statsmodels.api as sm
from statsmodels.api import OLS
import random
import statsmodels.formula.api as smf
import scipy as sp
import scipy.stats
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from pandas.tools.plotting import scatter_matrix, radviz
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

sns.set_style('whitegrid')
sns.set_context('notebook')

# Data loading

In [220]:
# Load the data into a dataframe 
#can_master = pd.read_csv('can-1996-2016.csv')
#cmte_master =pd.read_csv('cm-1996-2014.csv') 
#opex=pd.read_csv('opex-2004-2014.csv')
#oth=pd.read_csv('oth-1996-2014.csv')
#exp=pd.read_csv('expresults_rev.csv')
#election_spend=pd.read_csv('catall.csv')
#results = pd.read_csv('candr.csv')
#catsort= pd.read_csv('catsort_rev.csv')
cost_vote = pd.read_csv('combined_all.csv')

In [221]:
cost_vote['total_votes']=cost_vote['total_votes'].fillna(0)
cost_vote['totalspend']=cost_vote['totalspend'].fillna(0)
cost_vote['incumbent']=cost_vote['incumbent'].fillna(0)
cost_vote['party']=cost_vote['party'].fillna('None')
cost_vote = cost_vote.drop(['costpervote'], axis=1)
cost_vote['totalspend']=pd.to_numeric(cost_vote['totalspend'],errors='coerce')
print(cost_vote[pd.to_numeric(cost_vote['totalspend'], errors = 'coerce').isnull()])
cost_vote['total_votes']=pd.to_numeric(cost_vote['total_votes'],errors='coerce')
print(cost_vote[pd.to_numeric(cost_vote['total_votes'], errors = 'coerce').isnull()])
cost_vote['cycle']=pd.to_numeric(cost_vote['cycle'],errors='coerce')
print(cost_vote[pd.to_numeric(cost_vote['cycle'], errors = 'coerce').isnull()])
cost_vote = cost_vote[cost_vote.total_votes != 0]
cost_vote.loc[cost_vote['totalspend'] == 0]
cost_vote = cost_vote.rename(columns = {'ge_winner_indicator':'winner'})
cost_vote['costvote'] = cost_vote['totalspend']/cost_vote['total_votes']

Empty DataFrame
Columns: [cand_id, cycle, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, state_abbreviation, district, incumbent, party, primary_votes, runoff_votes, general_votes, ge_runoff_election_votes, ge_winner_indicator, total_votes, totalspend]
Index: []

[0 rows x 26 columns]
Empty DataFrame
Columns: [cand_id, cycle, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, state_abbreviation, district, incumbent, party, primary_votes, runoff_votes, general_votes, ge_runoff_election_votes, ge_winner_indicator, total_votes, totalspend]
Index: []

[0 rows x 26 columns]
Empty DataFrame
Columns: [cand_id, cycle, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, state_abbreviation, district, incumbent, party, primary_votes, runoff_votes, general_votes, ge_runoff_election_votes, ge_winner_indicator, total_votes, totalspend]
Index: []

[0 rows x 26 columns]


In [222]:
cost_vote.head()

,cand_id,cycle,c1,c2,c3,c4,c5,c6,c7,c8,...,incumbent,party,primary_votes,runoff_votes,general_votes,ge_runoff_election_votes,winner,total_votes,totalspend,costvote
0,H0AK00089,2010,550.00,0.00,0.00,0.00,0.0,785.00,0.00,0.0,...,0,DEM,38673.0,0.0,77606.0,0.0,0,116279.0,220001.06,1.892010
1,H0AK01038,2010,1569.23,88.93,0.00,0.00,0.0,0.00,0.00,0.0,...,0,REP,24709.0,0.0,0.0,0.0,0,24709.0,169777.74,6.871089
2,H0AL00016,2010,19831.34,909.80,81.19,465.00,0.0,3177.55,2192.99,0.0,...,0,DEM,6672.0,0.0,0.0,0.0,0,6672.0,26957.87,4.040448
3,H0AL01030,2010,7054.34,5233.32,668.95,27082.28,1175.0,1295.21,583.75,0.0,...,0,REP,18725.0,0.0,0.0,0.0,0,18725.0,44377.00,2.369933
4,H0AL01030,2012,3480.00,27.72,381.09,7257.71,0.0,0.00,56.58,0.0,...,0,R,3854.0,0.0,0.0,0.0,0,3854.0,11203.10,2.906876


In [223]:
cost_vote['cycle'] = cost_vote['cycle'].astype(int).astype('str')

In [224]:
cost_vote['party'] = cost_vote['party'].astype('str')
cost_vote = cost_vote.replace("DEM","D")
cost_vote = cost_vote.replace("DEM ","D")
cost_vote = cost_vote.replace("REP","R")
cost_vote = cost_vote.replace("REP ","R")
R = cost_vote[cost_vote['party'] == "R"]
D = cost_vote[cost_vote['party'] == "D"]

In [225]:
testframes = [R, D]

Gen = pd.concat(testframes)

In [226]:
Gen = Gen.replace("W (Runoff)","0")
Gen = Gen.replace("WW","0")
Gen = Gen.replace("Dd","0")

In [227]:
Gen['race'] = Gen['cand_id'].astype(str).str[0]

In [228]:
Gen = Gen[Gen['race'] == "H"]

# Getting information for VA10

In [229]:
Gen = Gen.rename(columns = {'state_abbreviation':'state'})

In [230]:
Gen['state'].unique()

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
       'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'GU', 'VI', 'AS', 'WY'], dtype=object)

In [231]:
VA = Gen[Gen['state'].isin(['VA'])]

In [232]:
VA10 = Gen[Gen['district'].isin(['10'])]

In [150]:
VA10.head(20)

,cand_id,cycle,c1,c2,c3,c4,c5,c6,c7,c8,...,party,primary_votes,runoff_votes,general_votes,ge_runoff_election_votes,winner,total_votes,totalspend,costvote,race
127,H0CA10180,2010,283.31,100.87,26.44,35.00,0.0,357.64,0.00,0.0,...,R,25331.0,0.0,88512.0,0.0,0,113843.0,26169.26,0.229871,H
128,H0CA10230,2010,13666.37,0.00,0.00,700.00,0.0,0.00,0.00,0.0,...,R,21060.0,0.0,0.0,0.0,0,21060.0,16079.76,0.763521,H
151,H0CA19173,2012,308246.12,71262.90,364254.68,1876410.58,58000.0,0.00,0.00,0.0,...,R,45779.0,0.0,110265.0,0.0,1,156044.0,2847573.22,18.248527,H
152,H0CA19173,2014,218106.53,69075.68,334543.04,1118773.98,64550.0,0.00,0.00,0.0,...,R,44237.0,0.0,70582.0,0.0,1,114819.0,1852645.26,16.135354,H
330,H0FL08208,2012,22816.85,502.62,1226.00,0.00,0.0,0.00,660.00,0.0,...,R,0.0,0.0,164649.0,0.0,1,164649.0,1486363.94,9.027470,H
467,H0IL10294,2010,201677.67,1272.78,24788.88,170558.24,18135.0,69280.38,5370.28,0.0,...,R,16149.0,0.0,0.0,0.0,0,16149.0,506326.61,31.353434,H
468,H0IL10302,2010,556411.21,790.70,116738.91,855766.04,18724.0,0.00,25449.63,0.0,...,R,19691.0,0.0,109941.0,0.0,1,129632.0,2871943.35,22.154586,H
469,H0IL10302,2012,38938.80,0.00,10053.57,0.00,0.0,0.00,0.00,0.0,...,R,36647.0,0.0,130564.0,0.0,0,167211.0,4445882.23,26.588455,H
470,H0IL10302,2014,11819.39,0.00,3786.01,0.00,0.0,0.00,0.00,0.0,...,R,32124.0,0.0,95992.0,0.0,1,128116.0,3610551.02,28.181890,H
471,H0IL10336,2010,764.98,855.83,2257.85,13860.85,0.0,3127.39,186.09,0.0,...,R,7260.0,0.0,0.0,0.0,0,7260.0,48001.99,6.611844,H


## VA Republican

In [260]:
Rep = VA10[VA10['party'].isin(['R'])]

In [261]:
RepW = Rep[Rep['winner'].isin(['1'])]

In [263]:
Rep.describe()

,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,primary_votes,runoff_votes,general_votes,ge_runoff_election_votes,total_votes,totalspend,costvote
count,1.270000e+02,127.000000,127.000000,1.270000e+02,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,1.270000e+02,127.000000,127.000000,127.000000,127.0,127.000000,1.270000e+02,127.000000
mean,1.148069e+05,3309.417638,31462.977717,1.249802e+05,5471.952205,14464.328425,2674.223622,78.740157,24.141496,96.850394,634.043307,92.768504,4.238349e+05,19295.031496,814.889764,73699.102362,0.0,93809.023622,7.218526e+05,21.359059
std,2.426591e+05,10870.903776,91571.977798,4.246735e+05,15690.132398,52209.903699,8433.764962,887.356509,191.622120,649.167600,3528.620099,759.917865,8.594884e+05,19935.376505,3894.701034,77869.892687,0.0,91787.361889,1.014073e+06,50.532532
min,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-7.782100e+02,0.000000,0.000000,0.000000,0.0,218.000000,0.000000e+00,0.000000
25%,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.440750e+02,2202.000000,0.000000,0.000000,0.0,7339.500000,2.019431e+04,1.122672
50%,1.001320e+03,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.568103e+04,11856.000000,0.000000,69996.000000,0.0,91981.000000,3.452670e+05,4.799941
75%,8.247940e+04,443.895000,9354.620000,1.289905e+04,0.000000,2804.020000,634.465000,0.000000,0.000000,0.000000,0.000000,0.000000,5.246949e+05,32331.000000,0.000000,132496.000000,0.0,160857.000000,9.442626e+05,15.876822
max,1.141098e+06,71262.900000,505319.320000,2.707270e+06,98520.000000,478942.640000,70120.410000,10000.000000,1549.260000,5000.000000,34500.000000,8450.000000,5.389727e+06,84426.000000,26975.000000,230471.000000,0.0,296474.000000,5.389727e+06,338.449885


In [265]:
Total_VoteRW = RepW['total_votes']
Total_SpendRW = RepW['totalspend']

def reg_m(Total_VoteRW,Total_SpendRW):
    Total_SpendRW=np.array(Total_SpendRW).T
    Total_SpendRW=sm.add_constant(Total_SpendRW)
    model=sm.OLS(endog=Total_VoteRW,exog=Total_SpendRW)
    results=model.fit()
    return results

print (reg_m(Total_VoteRW,Total_SpendRW).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.085
Model:                            OLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     4.346
Date:                Sat, 09 Sep 2017   Prob (F-statistic):             0.0426
Time:                        01:30:13   Log-Likelihood:                -600.01
No. Observations:                  49   AIC:                             1204.
Df Residuals:                      47   BIC:                             1208.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       2.098e+05   1.19e+04     17.611      0.0

In [266]:
Total_VoteRW = RepW['total_votes']
Total_SpendRW = [RepW['c1'],RepW['c2'],RepW['c3'],RepW['c4'],RepW['c5'],RepW['c6'],RepW['c7'],RepW['c8'],RepW['c9'],RepW['c10'],RepW['c11'],RepW['c12'],RepW['c13']]

def reg_m(Total_VoteRW,Total_SpendRW):
    Total_SpendRW=np.array(Total_SpendRW).T
    Total_SpendRW=sm.add_constant(Total_SpendRW)
    model=sm.OLS(endog=Total_VoteRW,exog=Total_SpendRW)
    results=model.fit()
    return results

print (reg_m(Total_VoteRW,Total_SpendRW).summary())

                            OLS Regression Results                            
Dep. Variable:            total_votes   R-squared:                       0.658
Model:                            OLS   Adj. R-squared:                  0.559
Method:                 Least Squares   F-statistic:                     6.652
Date:                Sat, 09 Sep 2017   Prob (F-statistic):           4.71e-06
Time:                        01:30:15   Log-Likelihood:                -640.69
No. Observations:                  49   AIC:                             1303.
Df Residuals:                      38   BIC:                             1324.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
x1             0.1798      0.080      2.259      0.0

Confidence Interval for Democrats and Republicans in General Election

In [267]:
Total_VoteR = Rep['total_votes']

def mean_confidence_interval(Total_VoteR, confidence=0.95):
    a = 1.0*np.array(Total_VoteR)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(Total_VoteR, confidence=0.95))

(93809.023622047243, 77690.681489972194, 109927.36575412229)


In [268]:
costvoteR = Rep['costvote']

def mean_confidence_interval(costvoteR, confidence=0.95):
    a = 1.0*np.array(costvoteR)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(costvoteR, confidence=0.95))

(21.359059267197363, 12.485281660118805, 30.232836874275922)


Confidence Intervals for Democrats and Republicans who who in General Election 

In [269]:
Total_VoteRW = RepW['total_votes']

def mean_confidence_interval(Total_VoteRW, confidence=0.95):
    a = 1.0*np.array(Total_VoteRW)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(Total_VoteRW, confidence=0.95))

(190272.38775510204, 175007.90301655477, 205536.87249364931)


In [270]:
costvoteRW = RepW['costvote']

def mean_confidence_interval(costvoteRW, confidence=0.95):
    a = 1.0*np.array(costvoteRW)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

print (mean_confidence_interval(costvoteRW, confidence=0.95))

(7.8907461367409804, 5.8705848644854601, 9.9109074089965006)
